In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model

In [ ]:
train_data_dir = os.path.join('../input/final-data-rps/Final Data/Training')
test_data_dir = os.path.join('../input/final-data-rps/Final Data/Testing')
categories = ['none','rock', 'paper', 'scissors']
training_data=[]
testing_data = []
img_size= 150

#function to label training and testing data
def data_labeling(data_path, data):
    for category in categories:
        #labels: 0 for none, 1 for rock, 2 for paper, and 3 for scissors
        label = categories.index(category)

        category_path = os.path.join(data_path, category)
        for img in os.listdir(category_path):
            img_path = os.path.join(category_path, img)
            img_array = cv2.imread(img_path) #image is being converted into an array
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB )
            img_array = cv2.resize(img_array, (img_size, img_size))
            plt.imshow(img_array)
            plt.title(label)
            #final labeling of each image to its respective category
            data.append([img_array, label])
            
data_labeling(train_data_dir, training_data)
data_labeling(test_data_dir, testing_data)

In [ ]:
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
for features, labels in training_data:
    X_train.append(features)
    Y_train.append(labels)  
X_train=np.array(X_train)
Y_train=np.array(Y_train)
for features, labels in testing_data:
    X_test.append(features)
    Y_test.append(labels)  
X_test=np.array(X_test)
Y_test=np.array(Y_test)
X_train= X_train/255
X_test= X_test/255



print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
from tensorflow.keras import layers
from keras.models import Sequential

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
image_size2=(150,150,3)
vgg= VGG16(input_shape=image_size2, weights='imagenet',include_top=False)
vgg.output
for layers in vgg.layers:
    layers.trainable =False

x=Flatten()(vgg.output)
prediction=Dense(len(X_train), activation='softmax')(x)
model = tf.keras.Sequential([data_augmentation])
model= Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(optimizer ='adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history= model.fit(X_train,Y_train, epochs =6,validation_split= 0.1, batch_size=8)
model.summary()
model.evaluate(X_test,Y_test)
            

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred= vgg.predict(X_test)
y_pred_classes=[np.argmax(element) for element in y_pred]
print("Classification Report: \n",classification_report(Y_test,y_pred_classes))


In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
from random import choice
input_path = os.path.join('../input/my-input/manual input')
input_data=[]
icon_path = os.path.join('../input/rps-icons/images')
icon=[]
def process(data_path, data):
    for img in os.listdir(data_path):
            img_path = os.path.join(data_path, img)
            img_array = cv2.imread(img_path) #image is being converted into an array
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB )
            img_array = cv2.resize(img_array, (150,150))
            data.append(img_array)

In [ ]:
process(input_path,input_data)
process(icon_path, icon)
input_data=np.array(input_data)
icon =np.array(icon)

In [ ]:
def prediction(data):
    y_pred= model.predict(data)
    for i in range(len(data)):
        output= np.argmax(y_pred[i])
        fig= plt.figure(figsize=(7,7))
        fig.add_subplot(1,2,1)
        plt.imshow(data[i])
        plt.axis('off')
        plt.title("Player's move")
        comp_move= choice(['rock', 'paper', 'scissors'])
        fig.add_subplot(1,2,2)
        plt.imshow(icon[categories.index(comp_move)-1])
        plt.axis('off')
        plt.title("Computer's move")
        
prediction(input_data)

In [ ]:
model.save("ROCK_PAPER_SCISSORS.h5")